In [ ]:
import pandas as pd
import json
import os, sys
import pycountry_convert as pc


#using gross syntax for relative import; will be fixed with packaging and bundling
example_code = '../../roundtrip/example_code/'


# This is the relative path from the notebook to our vis extensions
vis_path = os.path.join(example_code, 'magics_source')

# Add the path so that the notebook can find the Roundtrip extension
module_path = os.path.abspath(os.path.join(vis_path)) 
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext scatter
%load_ext table

## More Complex and Dynamic Workflows with `?` operator

### Automatic passing of javascript updatesbetween multiple views

We load data like we did before.

In [ ]:
dataset = pd.read_csv(os.path.join(example_code, 'data/Electricity_Production_By_Source.csv'))
vis_in = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

vis_in['Continent'] = vis_in.apply(getContinent, axis=1)

In the following cell you will see that the visualization developer provided an optional second argument to our scatterplot `out`. You will also see that this argument has a new operator `?`.

The `?` operator is provided by roundtrip to allow users the freedom to define Jupyter variables that they would like to act like 'passed by refrence' variables and auto-update when changed elsewhere in the notebook.

A visualization developer may disable this two way binding if they so choose. In the `scatter_plt` is it optional for both our variables `vis_in` and `out`.

In [ ]:
%scatter_plt vis_in ?subselect

Because `out` is bound with the `?` operator, selections made in the above plot will be automatically reflected in the following cell after it has been run at least once.

In [ ]:
%show_table ?subselect

In [ ]:
%scatter_plt ?subselect ?subsubselect

In [ ]:
%show_table ?subsubselect

### Automatic updating of RT visualizations on jupyter variables change

Here we define our dataset once again and visualize it.

In [ ]:
dataset = pd.read_csv(os.path.join(example_code, 'data/Electricity_Production_By_Source.csv'))
climate_data = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

climate_data['Continent'] = climate_data.apply(getContinent, axis=1)

In [ ]:
%scatter_plt ?climate_data ?selection

In [ ]:
%show_table ?selection

To get rid of some of our "garbage" data programatically, we can filter out all the data assocaited with the `oth` Continent. Because we have specified `climate_data` to auto-update above, we will see the `scatter_plt` reload with our new `climate_data`.

In [ ]:
climate_data = climate_data[climate_data['Continent'] != 'OTH']

We can also do other interesting things like filter to only get data from Europe. These manipulations are not supported by the visualization but enabled through the magic of Roundtrip.

In [ ]:
climate_data = climate_data[climate_data['Continent'] == 'EU']